In [2]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
import copy
def grad_U(Ui, Yij, Vj, ai, bj, reg, eta, mu):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return eta * (reg * Ui - Vj*((Yij - (mu + np.dot(Ui, Vj) + ai + bj))))

def grad_V(Vj, Yij, Ui, ai, bj, reg, eta, mu):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return eta * (reg * Vj - Ui*((Yij - (mu + np.dot(Ui, Vj) + ai + bj))))

def grad_a(Ui, Yij, Vj, ai,bj, reg, eta, mu):
    """Calculate the bias term for U"""
    return eta * (reg * ai - (Yij - (mu + np.dot(Ui, Vj) + ai + bj)))

def grad_b(Ui, Yij, Vj, ai, bj, reg, eta, mu):
    """Calculate the bias term for V"""
    return eta * (reg * bj - (Yij - (mu + np.dot(Ui, Vj) + ai + bj)))

def get_err(U, V, Y, a, b, mu, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    errors = []
    for (user, movie, rating) in Y:
        err = (rating - (mu + np.dot(U[user - 1], V[movie - 1]) + a[user - 1] + b[movie - 1]))**2
        errors.append(err)
    return  np.sqrt(sum(errors))/len(Y)

    
def train_model_with_bias(M, N, K, eta, reg, mu,  Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    # Initialize the U and V matrices to random
    # values in [-0.5, 0.5]
    U = np.random.rand(M, K) - 0.5
    V = np.random.rand(N, K) - 0.5
    
    # Have one ai for every user, and one vj for every movie
    a = np.random.rand(M, 1) - 0.5
    b = np.random.rand(N, 1) - 0.5
    
    delta_01 = None
    delta = None
    n = 0
    prev_loss = get_err(U, V, Y, a, b, mu, reg)
    
    print(prev_loss)
    # Run for input number of epochs
    while n < max_epochs:
        # Implement early stopping
        if delta_01 != None and delta != None and delta < eps * delta_01:
            break
        # Shuffle X and Y in order to add stochasticity
        p = np.random.permutation(len(Y))
        Y = Y[p]
        # In each epoch, iterate through all the points and
        # update the U and V vectors
        # Need to update the U and V simultaneously, so need to create a temp variable.
        for (user, movie, rating) in Y:
            gradu = grad_U(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            gradv = grad_V(V[movie - 1], rating, U[user - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            grada = grad_a(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)
            gradb = grad_b(U[user - 1], rating, V[movie - 1], a[user - 1], b[movie - 1], reg, eta, mu)         
            U[user - 1] = U[user - 1] - gradu
            V[movie - 1] = V[movie - 1] - gradv
            a[user - 1] = a[user -1] - grada
            b[movie - 1] = b[movie -1] - gradb             
        if n == 0:
            curr_loss = get_err(U, V,Y, a, b, mu, reg)
            delta_01 = prev_loss - curr_loss
            prev_loss = curr_loss
        else:
            curr_loss = get_err(U, V,Y, a, b, mu, reg)
            print(curr_loss)
            delta = prev_loss - curr_loss
            prev_loss = curr_loss
        n += 1
    print(n)
    final_mse = get_err(U, V, Y, a,b, mu)
    return (U, V, a, b, final_mse)






In [3]:
train = np.loadtxt('um/all.dta').astype(int)

In [5]:
train

array([[     1,    185,   2160,      1],
       [     1,    490,   2160,      4],
       [     1,   1811,   2160,      3],
       ...,
       [458293,  17567,   1796,      1],
       [458293,  17604,   1745,      3],
       [458293,  17689,   1715,      3]])

In [6]:
indices = np.loadtxt('um/all.idx').astype(int)

In [7]:
y_probe = []
y_base = []
y_valid = []
y_hidden = []
y_qual = []

for i, idx in enumerate(indices):
    if idx == 1:
        y_base.append(train[i])
    elif idx == 2:
        y_valid.append(train[i])
    elif idx == 3:
        y_hidden.append(train[i])
    elif idx == 4:
        y_probe.append(train[i])
    else:
        y_qual.append(train[i])

In [8]:
M = max(train[:,0]).astype(int) # users
N = max(train[:,1]).astype(int) # movies

In [34]:
print("Factorizing with ", M, " users, ", N, " movies.")
k = 20
reg = 0.1
eta = 0.1 # learning rate
mu = np.mean(np.array(y_base + y_valid + y_hidden + y_probe)[:, 3])
U, V, a, b, rmse = train_model_with_bias(M, N, k, eta, reg, mu, np.array(y_base + y_valid + y_hidden + y_probe)[:, [0, 1, 3]])
V = V.T

Factorizing with  458293  users,  17770  movies.
[0.00012161]
[9.35081998e-05]
[9.30461058e-05]
[9.27270261e-05]
[9.27330291e-05]
5


In [15]:
U

array([[ 0.26694572,  0.18593633, -0.21908995, ..., -0.09893097,
         0.35584393, -0.12157863],
       [-0.09354178, -0.10360051, -0.01521018, ...,  0.03355135,
        -0.39683215, -0.19730886],
       [-0.09140521, -0.12679148,  0.14712753, ...,  0.14465797,
         0.28422754,  0.09028438],
       ...,
       [-0.03010121, -0.01638623, -0.10615417, ...,  0.03665387,
        -0.14651433,  0.03237452],
       [-0.00122643,  0.09361242, -0.09224936, ...,  0.04259903,
         0.01005817, -0.11262746],
       [-0.04548208, -0.11773063,  0.16672062, ..., -0.06638057,
        -0.05651275,  0.00096565]])

In [35]:
print("Test error is ", get_err(U, V.T,np.array(y_probe)[:, [0, 1, 3]], a, b, mu))

Test error is  [0.00081946]


In [36]:
test_preds = []
for (user, movie, rating) in np.array(y_qual)[:, [0, 1, 3]]:
    test_preds.append((mu + np.dot(U[user - 1], V.T[movie - 1]) + a[user - 1] + b[movie - 1])[0])

In [28]:
len(test_preds)

2749898

In [37]:
with open('test_preds2.txt', 'w') as f:
    for item in test_preds:
        f.write("%.4f\n" % item)